In [ ]:
%useLatestDescriptors
%use lets-plot
%use dataframe

# Load data

In [ ]:
import org.um.feri.ears.algorithms.gp.GPAlgorithmExecutor
import org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData
import org.um.feri.ears.util.gp_stats.GPProgramSolutionSimple

var dataFile = "C:\\Users\\marko\\UnityProjects\\GenIATraP_refactor\\GeneralTrainingEnvironmentForMAS\\evo_monitor\\src\\data\\multiConfigurationPrograssData.ser"
var progressData = GPAlgorithmMultiConfigurationsProgressData.deserializeState(dataFile)
System.out.println("Progress data loaded from file: " + progressData)

System.out.println("Data size: " +progressData.multiConfigurationProgressData.size)

# Prepare helper classes & functions

In [ ]:
class SelectedIndividual(
    var selectedConfiguration: Int,
    var selectedRun: Int,
    var selectedGeneration: Int,
    var bestWorstIndividual: Int, // 0 = best, 1 = worst
) {
    constructor() : this(0, 0, 0, 0)
}

In [ ]:
import org.um.feri.ears.individual.representations.gp.IndividualMatchResult

fun sumIndividualValues(individualMatchResult: IndividualMatchResult): Double {
    var sum = 0.0
    for (key in individualMatchResult.individualValues.keys) {
        sum += individualMatchResult.individualValues[key]!!
    }

    return sum
}

In [ ]:
import org.yaml.snakeyaml.util.Tuple

// Prepare data (best, worst and average individual rating per generation)
fun prepareData(selectedConfiguration: Int, selectedRun: Int, bestGenIndividualsRating: MutableList<GPProgramSolutionSimple>, worstGenIndividualsRating: MutableList<GPProgramSolutionSimple>, avgGenIndividualRatings: MutableList<Double>, bestGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>, worstGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>, avgGenIndividualAvgFitnesses: MutableList<Double>){

    var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
    var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

    for (genProgressData in multiRunProgressData.gensProgressData) {
        var bestGenIndividualRating = genProgressData.population[0]
        var worstIndividualRating = genProgressData.population[0]
        var avgGenIndividualRating = 0.0

        var bestGenIndividualAvgFitness = genProgressData.population[0]
        var worstIndividualAvgFitness = genProgressData.population[0]
        var avgGenIndividualAvgFitness = 0.0

        for (solution in genProgressData.population) {
            if (solution.finalIndividualFitness.additionalValues["Rating"]!! < bestGenIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                bestGenIndividualRating = solution
            }

            if (solution.finalIndividualFitness.additionalValues["Rating"]!! > worstIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                worstIndividualRating = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) < sumIndividualValues(bestGenIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                bestGenIndividualAvgFitness = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) > sumIndividualValues(worstIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                worstIndividualAvgFitness = solution
            }

            avgGenIndividualRating += solution.finalIndividualFitness.additionalValues["Rating"]!!
            avgGenIndividualAvgFitness += sumIndividualValues(solution.finalIndividualFitness.avgMatchResult)
        }

        bestGenIndividualsRating.add(bestGenIndividualRating)
        worstGenIndividualsRating.add(worstIndividualRating)

        avgGenIndividualRating /= genProgressData.population.size
        avgGenIndividualRatings.add(avgGenIndividualRating)

        bestGenIndividualAvgFitnesses.add(bestGenIndividualAvgFitness)
        worstGenIndividualAvgFitnesses.add(worstIndividualAvgFitness)

        avgGenIndividualAvgFitness /= genProgressData.population.size
        avgGenIndividualAvgFitnesses.add(avgGenIndividualAvgFitness)
    }
}

# Visualize data (line plot) - Best, worst and average individual rating per generation

In [ ]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

// Prepare data
val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualRatings = mutableListOf<Double>()

val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

val generations = mutableListOf<Int>()

for (i in 0 until bestGenIndividualsRating.size) {
    generations.add(i)
}

// Display data
val dfRating = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + worstGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + avgGenIndividualRatings,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

val dfAvgFitness = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + worstGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + avgGenIndividualAvgFitnesses,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

// The plot variable stores a plot with the temperature values for each city
val linePlotRating =
    letsPlot(dfRating.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,400) + geomPoint(shape = 15) + geomLine() + ggtitle("Best Generation Rating")

val linePlotAvgFitness =
    letsPlot(dfAvgFitness.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,400) + geomPoint(shape = 15) + geomLine() + ggtitle("Best Generation Average fitness")

val plots = listOf(linePlotRating, null, linePlotAvgFitness, null)
gggrid(plots, ncol = 2)


# Visualize data (radar plot) - Comparison of individuals from different generations, runs & configurations

In [ ]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 9, 0))
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualRatings = mutableListOf<Double>()

    val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

val fitnessKeys = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    for (key in individual.finalIndividualFitness.avgMatchResult.individualValues.keys) {
        if (key !in fitnessKeys) {
            fitnessKeys.add(key)
        }
    }
}

// Order fitnessKeys alphabetically
fitnessKeys.sort()

// Build labels for visualization
val labels = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    labels.addAll(fitnessKeys)
}

val datasets = mutableListOf<MutableList<Double>>()

// Build datasets for visualization
for (individual in selectedGPAlgorithmSolutionsSimple) {
    val dataset = mutableListOf<Double>()
    for (key in fitnessKeys) {
        if(individual.finalIndividualFitness.avgMatchResult.individualValues[key] == null) {
            dataset.add(0.0)
            continue
        }
        dataset.add(-individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!)
    }
    datasets.add(dataset)
}

var fitnessKeysIndexed = mutableListOf<Int>()
for (i in 0 until fitnessKeys.size) {
    fitnessKeysIndexed.add(i + 1)
}

var individualValueIdsMapped = mutableListOf<Int>()
var individualMapped = mutableListOf<String>()
var index = 0
for(dataset in datasets) {
    individualValueIdsMapped.addAll(fitnessKeysIndexed)
    individualMapped.addAll(List(fitnessKeys.size) { "Individual " + (index + 1) })
    index++
}

// Build map for visualization
val selectedIndividualsData = mapOf(
    "IndividualValueLabels" to labels,
    "IndividualValueId" to individualValueIdsMapped,
    "Individual" to individualMapped,
    "IndividualValues" to datasets.map { it }.toMutableList().flatten()
)

// Map each fitnessKey to a map 1 -> "fitnessKey1", 2 -> "fitnessKey2", ...
var labelsIndexed = fitnessKeys.mapIndexed { index, s -> index to s }.toMap().mapKeys { it.key + 1 }

letsPlot(selectedIndividualsData) +
        geomArea(flat = true) {   // <-- flat. I.e., do not transform segments to curves
            x = "IndividualValueId"; y = "IndividualValues"
        } +
        geomPoint() { x = "IndividualValueId"; y = "IndividualValues"; color="Individual" } +
        scaleXDiscrete(labels = labelsIndexed) +
        coordPolar()

# Selected Individual Details

In [ ]:
import org.jetbrains.letsPlot.intern.Plot

// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 0, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 9, 0))
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualRatings = mutableListOf<Double>()

    val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

var plots = mutableListOf<Plot>()

var index = 0
for (individual in selectedGPAlgorithmSolutionsSimple) {
    var nodeCountsData = mapOf(
        "NodeName" to individual.nodeCounts.keys.toList(),
        "NodeCount" to individual.nodeCounts.values.toList()
    )

    val blankTheme = theme(line=elementBlank(), axis=elementBlank())
    val (w, h) = 800 to 400
    val p = letsPlot(nodeCountsData) + ggsize(w,h) + blankTheme

    val plot = p + geomPie(stat = Stat.identity,
            size = 20, stroke = 1, color = "white", hole = 0.5) { slice = "NodeCount"; fill = "NodeName" } +
            blankTheme +
            scaleFillBrewer(palette = "Set1") + ggtitle("Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId)

    System.out.println(
        "Individual: " + "(Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId + ")\nChangesCount: " + individual.changesCount + "\nFunctionNodesCount: " + individual.functionNodes + "\nTerminalNodesCount: " + individual.terminalNodes + "\nTreeSize: " + individual.treeSize + "\nTreeDepth: " + individual.treeDepth + "\nFitness: " + (individual.finalIndividualFitness.additionalValues["Rating"].toString() + ", " + individual.finalIndividualFitness.additionalValues["StdDeviation"]) + "\nIndividualAvgFitness: " + sumIndividualValues(individual.finalIndividualFitness.avgMatchResult) + "\nIndividualAvgFitnessValues: " + individual.finalIndividualFitness.avgMatchResult.individualValues.map { it.key + ": " + it.value }.toString()
        + "\n\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"
    )

    index++

    plots.add(plot)
}

gggrid(plots, ncol = 2)

# Convergence graphs

In [ ]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData

val n = 3
val Generation = mutableListOf<Int>()
val Rating = mutableListOf<Double>()

val convergenceDataGraph = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData;
for (i in 0 until convergenceDataGraph.size) {
    Generation.addAll(List(n) {i + 1})
    val rating = Math.abs(convergenceDataGraph[i].finalIndividualFitness.additionalValues["Rating"]!!)
    Rating.addAll(listOf(rating - (2*convergenceDataGraph[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*convergenceDataGraph[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
}

val data = mapOf<String, Any>(
    "Generation" to Generation,
    "Rating" to Rating
)

letsPlot(data) { x = "Generation"; y = "Rating"}  + geomBoxplot(whiskerWidth = 0.2) + ggtitle("Convergence graph (Tournament between generation best individuals)")